In [1]:
import pandas as pd
import numpy as np 
import re
import src.funcionesclean as op
import warnings
warnings.filterwarnings("ignore")

# Data Cleaning
## Importamos la tabla con la que vamos a trabajar
### Y vemos que tenemos dentro

In [2]:
hotel = op.read('input/Hotel_Reviews.csv')
hotel = pd.DataFrame(hotel)

In [3]:
hotel.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [4]:
hotel.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

#### Revisamos nulos

In [5]:
hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [6]:
hotel.isnull().sum()

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
dtype: int64

## Limpieza de las columnas que necesitaremos 


### Reviewer_nationality


#### Quitamos los espacios de antes y despues de cada pais. 

In [7]:
hotel['Reviewer_Nationality'] = hotel['Reviewer_Nationality'].apply(op.espacios)
pd.unique(hotel['Reviewer_Nationality'])

array(['Russia', 'Ireland', 'Australia', 'United Kingdom', 'New Zealand',
       'Poland', 'Belgium', 'Norway', 'France', 'Italy', 'Canada',
       'Netherlands', 'Germany', 'United States of America', 'Hungary',
       'Israel', 'Switzerland', 'India', 'United Arab Emirates',
       'Malaysia', 'Denmark', 'South Africa', 'Rwanda', 'Finland',
       'Portugal', 'Luxembourg', 'Spain', 'Malta', 'China', 'Turkey',
       'Ukraine', 'Gibraltar', 'Argentina', 'Abkhazia Georgia',
       'Isle of Man', 'Sweden', 'Croatia', 'Saudi Arabia', 'Lithuania',
       'Slovenia', 'Panama', 'Austria', 'Brazil', 'Taiwan', 'Iceland',
       'Iran', 'Liechtenstein', 'Chile', 'Hong Kong',
       'United States Minor Outlying Islands', 'Morocco', 'Oman',
       'Lebanon', 'Qatar', 'Nigeria', 'Romania', 'Czech Republic',
       'Greece', 'Thailand', 'Singapore', 'Guernsey', 'Latvia', 'Namibia',
       'Japan', 'Egypt', 'Slovakia', 'South Korea', 'Kuwait', 'Bahrain',
       'Jersey', 'Jordan', 'Bulgaria', 'Cyp

### Creamos una columna para el mes y otra para el año. 

In [8]:
hotel['Month'] = hotel['Review_Date'].str.extract('(\d+)')
hotel['Year'] = hotel['Review_Date'].str.extract('(\d{4})')
hotel['Month'] = hotel['Month'].apply(op.mes)
hotel.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Month,Year
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,08Agosto,2017
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,08Agosto,2017
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,07Julio,2017
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,07Julio,2017
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,07Julio,2017


## Segun el Address del hotel sacamos el pais donde se encuentra. 

In [9]:
pd.unique(hotel['Hotel_Address'])

array([' s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands',
       '1 15 Templeton Place Earl s Court Kensington and Chelsea London SW5 9NB United Kingdom',
       '1 2 Serjeant s Inn Fleet Street City of London London EC4Y 1LL United Kingdom',
       ...,
       'Wilton Place Knightsbridge Westminster Borough London SW1X 7RL United Kingdom',
       'Wrights Lane Kensington and Chelsea London W8 5SP United Kingdom',
       'Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria'],
      dtype=object)

In [10]:
hotel['Hotel_Country'] = hotel['Hotel_Address'].str.extract('(\w+$)')

In [11]:
pd.unique(hotel['Hotel_Country'])

array(['Netherlands', 'Kingdom', 'France', 'Spain', 'Italy', 'Austria'],
      dtype=object)

In [12]:
#Incluyo una funcion lambda ya que era uno de los requisitos del proyecto. 
hotel['Hotel_Country'] = hotel['Hotel_Country'].apply(lambda x : 'United Kingdom' if x == 'Kingdom' else x)
pd.unique(hotel['Hotel_Country'])

array(['Netherlands', 'United Kingdom', 'France', 'Spain', 'Italy',
       'Austria'], dtype=object)

In [13]:
hotel_clean = hotel
hotel_clean.to_csv("output/hotel_clean.csv")

# Datos en España
### Vamos a crear la tabla de datos para los hoteles que estan en España. 

In [14]:
hotel_esp = hotel_clean[(hotel_clean["Hotel_Country"]== "Spain")]

In [15]:
hotel_esp.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Month,Year,Hotel_Country
117428,234 Diputacio Eixample 08007 Barcelona Spain,27,7/2/2017,7.8,Sansi Diputacio,United States of America,First room the air conditioner wasn t working...,60,338,Restaurant across the street opens at 6am and...,27,1,6.3,"[' Leisure trip ', ' Couple ', ' Double Room '...",32 days,41.388223,2.164455,07Julio,2017,Spain
117429,234 Diputacio Eixample 08007 Barcelona Spain,27,4/14/2017,7.8,Sansi Diputacio,Israel,No Negative,0,338,The value for money was the greatest I ve see...,54,14,9.6,"[' Leisure trip ', ' Couple ', ' Double Room '...",111 day,41.388223,2.164455,04Abril,2017,Spain
117430,234 Diputacio Eixample 08007 Barcelona Spain,27,4/5/2017,7.8,Sansi Diputacio,Sweden,Seems like the cleaning personal misunderstoo...,59,338,I like the area its close to passeig de graci...,22,6,6.7,"[' Leisure trip ', ' Family with young childre...",120 day,41.388223,2.164455,04Abril,2017,Spain
117431,234 Diputacio Eixample 08007 Barcelona Spain,27,7/17/2016,7.8,Sansi Diputacio,United Kingdom,Having a Pool would have been better I felt t...,68,338,Location was good as very central,8,2,5.8,"[' Leisure trip ', ' Couple ', ' Double Room '...",382 day,41.388223,2.164455,07Julio,2016,Spain
117432,234 Diputacio Eixample 08007 Barcelona Spain,27,6/30/2016,7.8,Sansi Diputacio,Hungary,to be asked to forward my booking com booking...,41,338,location smiley staff,4,3,5.8,"[' Business trip ', ' Solo traveler ', ' Doubl...",399 day,41.388223,2.164455,06Junio,2016,Spain


## Revisamos la columna de Negative_Reviews para delimitar todas las reviews en cinco valores segun lo que ponga en el comentario. 
#### 'Aire Acondicionado','Comentario positivo','Suciedad','Antiguedad' ,'Habitaciones', 'Utilities' ,'Comidas'

In [16]:
hotel_esp.Negative_Review.head()

117428     First room the air conditioner wasn t working...
117429                                          No Negative
117430     Seems like the cleaning personal misunderstoo...
117431     Having a Pool would have been better I felt t...
117432     to be asked to forward my booking com booking...
Name: Negative_Review, dtype: object

In [17]:
hotel_esp['Negative_Review2'] = hotel_esp['Negative_Review'].apply(op.neg)

In [18]:
hotel_esp['Negative_Review2'].head()

117428     Aire Acondicionado
117429    Comentario positivo
117430               Suciedad
117431     Aire Acondicionado
117432             Antiguedad
Name: Negative_Review2, dtype: object

### Creamos la tabla para los datos de españa con la que trabajaremos en el Storytelling

In [19]:
hotel_esp.to_csv("output/hotel_esp.csv")